# Importação das bibliotecas necessárias

In [17]:
import sys
import os
sys.path.append('/home/jovyan/work')

from utils.AlinharETL import AlinharETL

# Crie uma instância da classe AlinharETL

In [18]:
# Crie uma instância da classe
bucket = "bronze"
datamart = "SiteTCUAcordaos"
extrator_bronze = AlinharETL(bucket,datamart)

2024-10-09 11:02:13,481 - INFO - Iniciando Sessão Spark.


# Leitura da tabela

In [19]:
df_compliance_tcu  = extrator_bronze.criar_view_temporaria('bronze/TCU/Acordaos','compliance_tcu')

2024-10-09 11:02:14,162 - INFO - Aguarde... Criando View (bronze/TCU/Acordaos)
2024-10-09 11:02:19,611 - INFO - View criada com sucesso


# Tratamentos na tabela

In [20]:
sql_query = """
    select 
        ROW_NUMBER() OVER (ORDER BY KEY) AS id,
        KEY as key_acordao,
        NUMACORDAO as numacordao,
        to_date(DATASESSAO, 'dd/MM/yyyy') AS datasessao,
        COLEGIADO as colegiado,
        ASSUNTO as assunto,
        NUMATA as numata,
        PROC as proc,
        CONCAT('https://pesquisa.apps.tcu.gov.br/push/processo?numero=', TRANSLATE(SUBSTRING_INDEX(IFNULL(TRIM(PROC), '00000000000'), ' ', 1), '.-/','')) as proc_acordao,
        RELATOR as relator,
        SUMARIO as sumario, 
        'Não Informado' as sistema_s,
        ENTIDADE as entidade,
        ANOACORDAO as anoacordao,
        TIPO as tipo  
    from compliance_tcu
"""

In [21]:
df_compliance_tcu = extrator_bronze.carregar_dados_delta(sql_query)

2024-10-09 11:02:26,259 - INFO - Aguarde... Executando Query (Delta)
2024-10-09 11:02:26,260 - INFO - 
    select 
        ROW_NUMBER() OVER (ORDER BY KEY) AS id,
        KEY as key_acordao,
        NUMACORDAO as numacordao,
        to_date(DATASESSAO, 'dd/MM/yyyy') AS datasessao,
        COLEGIADO as colegiado,
        ASSUNTO as assunto,
        NUMATA as numata,
        PROC as proc,
        CONCAT('https://pesquisa.apps.tcu.gov.br/push/processo?numero=', TRANSLATE(SUBSTRING_INDEX(IFNULL(TRIM(PROC), '00000000000'), ' ', 1), '.-/','')) as proc_acordao,
        RELATOR as relator,
        SUMARIO as sumario, 
        'Não Informado' as sistema_s,
        ENTIDADE as entidade,
        ANOACORDAO as anoacordao,
        TIPO as tipo  
    from compliance_tcu

2024-10-09 11:02:26,316 - INFO - Query (Delta) executada com sucesso


In [22]:
df_compliance_tcu.printSchem

+---+--------------------+----------+----------+--------------+--------------------+-------+--------------------+--------------------+--------------------+--------------------+-------------+--------------------+----------+-------+--------------------+
| id|         key_acordao|numacordao|datasessao|     colegiado|             assunto| numata|                proc|        proc_acordao|             relator|             sumario|    sistema_s|            entidade|anoacordao|   tipo|      dt_atualizacao|
+---+--------------------+----------+----------+--------------+--------------------+-------+--------------------+--------------------+--------------------+--------------------+-------------+--------------------+----------+-------+--------------------+
|  1|  ACORDAO-COMPLETO-1|        98|1992-03-12|Segunda Câmara|Tomada  de  Conta...|08/1992|      349.005/1991-5|https://pesquisa....|  BENTO JOSÉ BUGARIN|Tomada de Contas ...|Não Informado|                NULL|      1992|DECISÃO|2024-10-09 11:

# Gravação no datalake

In [6]:
extrator_bronze.caminho_tabela_delta = 's3a://silver/Tcu/TcuAcordaos'

In [7]:
extrator_bronze.salvar_delta(df_compliance_tcu, 'overwrite')

2024-09-25 19:17:52,615 - INFO - Aguarde... Persistindo dados (overwrite)
2024-09-25 19:18:32,809 - INFO - Dados persistidos com sucesso
2024-09-25 19:18:32,809 - INFO - s3a://silver/Tcu/TcuAcordaos
2024-09-25 19:18:32,810 - INFO - Aguarde... Realizando optimize
2024-09-25 19:18:36,553 - INFO - Optimize executado com sucesso.
2024-09-25 19:18:36,554 - INFO - Aguarde... Realizando vacuum
2024-09-25 19:18:50,045 - INFO - Vacuum executado com sucesso.


# Encerra sessão spark

In [8]:
extrator_bronze.parar_sessao()

2024-09-25 19:18:50,605 - INFO - Sessão Spark finalizada.
